In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import copy

In [ ]:
import autoreload
import evotsc
autoreload.reload(evotsc)

In [ ]:
# Population-level constants
intergene = 1000
interaction_dist = 2500
beta_A = 0.0
beta_B = 0.25

In [ ]:
def plot_expr(indiv, plot_title, plot_name):
    
    temporal_expr, fitness = indiv.evaluate()
    
    nb_genes, nb_steps = temporal_expr.shape
    
    colors = mpl.cm.get_cmap('viridis', nb_genes)(range(nb_genes))
    
    plt.figure(figsize=(9, 8), dpi=200)
    
    plt.ylim(-0.1, 2.1)
    
    for gene in range(nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[indiv.genes[gene].id, :],
                 linestyle=linestyle,
                 color=colors[indiv.genes[gene].id],
                 label=f'Gene {indiv.genes[gene].id}')
    
    plt.grid(linestyle=':')
    plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')
    
    plt.legend(loc='center right')
    plt.title(plot_title + f' fitness: {fitness:.5}')
        
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')

In [ ]:
def plot_expr_AB(indiv, plot_title, plot_name):
    
    (temporal_expr_A, temporal_expr_B), fitness = indiv.evaluate(beta_A=beta_A, beta_B=beta_B)
        
    colors_A = ['tab:green', 'tab:green', 'tab:red'] # A & B or A are valid
    colors_B = ['tab:green', 'tab:red', 'tab:green'] # A & B or B are valid
    
    plt.figure(figsize=(9, 6), dpi=200)

    ## First subplot: environment A
    plt.subplot(2, 1, 1)
    plt.ylim(-0.1, 2.1)
    
    for gene in range(indiv.nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr_A[indiv.genes[gene].id, :],
                 linestyle=linestyle,
                 color=colors_A[indiv.genes[gene].gene_type],
                 label=f'Gene {indiv.genes[gene].id}')
    
    plt.grid(linestyle=':')
    #plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')
    
    #plt.legend(loc='center right')
    plt.title('Environment A')
    
    ## Second subplot: environment B
    plt.subplot(2, 1, 2)
    plt.ylim(-0.1, 2.1)
    
    for gene in range(indiv.nb_genes):
        linestyle = 'solid' if indiv.genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr_B[indiv.genes[gene].id, :],
                 linestyle=linestyle,
                 color=colors_B[indiv.genes[gene].gene_type],
                 label=f'Gene {indiv.genes[gene].id}')
    
    plt.grid(linestyle=':')
    plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')
    
    #plt.legend(loc='center right')
    plt.title('Environment B')
    
    ## Final stuff
    plt.suptitle(f'{plot_title} fitness: {fitness:.5}')
    
    plt.tight_layout()

        
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')

In [ ]:
init_genes = evotsc.Gene.generate(intergene=intergene, nb_genes=20, default_basal_expression=1.0)

In [ ]:
init_indiv = evotsc.Individual(genes=init_genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=0.3,
                               nb_eval_steps=51)

In [ ]:
mutation = evotsc.Mutation(intergene_mutation_prob=0.0,
                           intergene_mutation_var=0.0,
                           inversion_prob=1e-2)

In [ ]:
plot_expr_AB(indiv=init_indiv, plot_title='best generation 0', plot_name='best generation 0')

In [ ]:
population = evotsc.Population(init_indiv=init_indiv, nb_indivs=1000, mutation=mutation)

In [ ]:
step = 50

best_indivs = []
gen = 0
for i in range(10):
    best_indivs += population.evolve(step, beta_A, beta_B)
    gen += step
    plot_expr_AB(best_indivs[-1], f'best generation {gen}', f'best generation {gen}.png')